In [1]:
import dash
import numpy as np
from PIL import Image
from dash import dcc, html, Input, Output
import dash.dependencies as dd
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#for displaying the logo image
import base64

In [2]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "INTERACTIVE DATA VISUALIZATION"

colors = {
    'background': '#DADADA',
    'text': '#111111'
}

# logo image
image_filename = 'C:/Users/Rnowa/Documents/interactive-data-visualization-webapp/logo_square.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# bottom info text elements
text1 = """
The price of the LUNA cryptocurrency running on the Terra blockchain has been falling since april. Even more recently, the token has suffered a massive dip resulting in the price falling to aproximately 0.00017 USD per coin. The chart above shows the price of the LUNA cryptocurrency.
"""
text2 = """
This chart shows the frequency of the given keywords on twitter (data as of May 14, 2022). The keywords are classified according to the positive and the negative context in the emotional sense.
"""
text3 = """
These are the keywords presented in the wordcloud. The bigger the word is displayed, the higher the frequency of its occurrence on twitter (data from May 14, 2022). The keywords are classified according to the positive and the negative context in the emotional sense.
"""
# data for the wordcloud
pos_common = ['top', 'price', 'week', 'next', 'analyst', 'target', 'like', 'burn', 'buy', 'back', 'ust', 'terra', 'money', 'crypto', 'great', 'would', 'yes', 'good', 'market', 'please', 'think', 'let', 'tokens', 'value', 'make', 'happy', 'get', 'way', 'better', 'people', 'still', 'worth', 'could', 'save', 'much', 'new', 'one', 'thank', 'help', 'know', 'hope', 'well', 'btc', 'lost', 'see', 'may', 'come', 'going', 'ecosystem']

neg_common = ['ust', 'buy', 'people', 'crypto', 'terra', 'price', 'bitcoin', 'burn', 'would', 'still', 'back', 'bought', 'kwon', 'see', 'time', 'coins', 'market', 'btc', 'going', 'go', 'lost', 'goes', 'everyone', 'collapse', 'happen', 'crash', 'get', 'think', 'us', 'moon', 'next', 'never', 'make', 'let', 'right', 'got', 'first', 'holders', 'supply', 'new', 'buying', 'way', 'money', 'said', 'one', 'current', 'take', 'token', 'na']

# data for the barplot
pos = [('top', 31),
 ('price', 30),
 ('week', 30),
 ('next', 27),
 ('analyst', 23),
 ('target', 23),
 ('like', 23),
 ('burn', 20),
 ('buy', 19),
 ('back', 18),
 ('ust', 16),
 ('terra', 12),
 ('money', 11),
 ('crypto', 10),
 ('great', 10),
 ('would', 10),
 ('yes', 9),
 ('good', 9),
 ('market', 9),
 ('think', 8),
 ('let', 8),
 ('tokens', 8),
 ('please', 8),
 ('value', 8),
 ('make', 7),
 ('happy', 7),
 ('get', 7),
 ('way', 7),
 ('better', 7),
 ('people', 7),
 ('still', 7),
 ('worth', 7),
 ('could', 7),
 ('save', 6),
 ('much', 6),
 ('new', 6),
 ('one', 6),
 ('thank', 6),
 ('help', 6),
 ('know', 6),
 ('hope', 6),
 ('well', 6),
 ('btc', 6),
 ('lost', 5),
 ('see', 5),
 ('may', 5),
 ('come', 5),
 ('going', 5),
 ('ecosystem', 5)]

neg = [('ust', 25),
 ('buy', 20),
 ('people', 18),
 ('crypto', 17),
 ('terra', 17),
 ('price', 15),
 ('bitcoin', 15),
 ('burn', 13),
 ('would', 13),
 ('still', 12),
 ('back', 11),
 ('bought', 10),
 ('kwon', 10),
 ('see', 10),
 ('time', 9),
 ('coins', 9),
 ('market', 9),
 ('btc', 9),
 ('go', 9),
 ('lost', 9),
 ('goes', 9),
 ('moon', 8),
 ('everyone', 8),
 ('collapse', 8),
 ('happen', 8),
 ('crash', 8),
 ('going', 8),
 ('get', 8),
 ('think', 8),
 ('us', 8),
 ('next', 7),
 ('buying', 7),
 ('never', 7),
 ('make', 7),
 ('let', 7),
 ('right', 7),
 ('got', 7),
 ('first', 7),
 ('supply', 6),
 ('new', 6),
 ('way', 6),
 ('money', 6),
 ('said', 6),
 ('one', 6),
 ('current', 6),
 ('holders', 6),
 ('take', 6),
 ('token', 5),
 ('na', 5)]

# from the pos, neg lists of tuples to DataFrames
d_pos = dict(pos)
d_neg = dict(neg)

keywords_pos = d_pos.keys()
freq_pos = d_pos.values()

keywords_neg = d_neg.keys()
freq_neg = d_neg.values()

pd_pos = pd.DataFrame()
pd_neg = pd.DataFrame()

pd_pos['keywords'] = keywords_pos
pd_pos['freq'] = freq_pos

pd_neg['keywords'] = keywords_neg
pd_neg['freq'] = freq_neg

In [3]:
# root node
app.layout = html.Div(
    style={
        'height': '98vh',
        'backgroundColor': colors['background']
        },
    children=[
        
        #logo element
    html.Div(
        [html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                 style={
                     'padding-left': '4.1%',
                     'padding-top': '0.7%',
                     'height':'5.5%',
                     'width':'5.5%',
                     'float': 'left',
                     'clear': 'left'
                     }
                 )
            ]
        ),
    #title element
    html.H1(
        children='INTERACTIVE DATA VISUALIZATION WEBAPP',
        style={
            'margin': 'auto',
            'width': '70%',
            'padding-top': '2.5vh',
            'margin-bottom': '2.5%',
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
        
    # THE DATA VISUALIZATION element
    html.Div(style={
        #'border-style': 'solid',
        'border-color': '#7FDBFF',
        'margin-left': 'auto',
        'width': '70%',
        'height': '60vh',
        'padding': '10px',
        'float': 'left',
        'clear': 'left'
            },
        children=html.Div(id='visualization',
                          style={'padding-left': '5%',})
        ),
        
    # INSTRUCTIONS element
    html.Div(style={
            'padding-right': '3%'
        },
             
        children=[
            html.Div(style={
                        'margin': 'auto',
                        'width': '20%',
                        'padding': '10px',
                        'color': colors['text'],
                        'fontSize': '15px',
                        'textAlign': 'left',
                        'float': 'right',
                        'clear': 'right'
                    }, 
                     children=[
                        html.H2('Instructions'),
                        html.P('1. For information about the data, refer to the text below the visualization.'),
                        html.P('2. Use the slider to change the displayed visualization.'),
                        html.P('3. Display the keyword barplot for positive and negative tweets.'),
                        html.P('4. Display the keywords as a positive or negative wordcloud.')
                    ]
                ),

            # INTERACTIVE CONTROLS element
            html.Div(style={
                        'float': 'right',
                        'clear': 'right'
                },
                    #slider
                    children=[
                    html.Div(
                    dcc.Slider(1, 3, 1,
                           value=1,
                           id='my-slider'
                            ),
                    style={
                        'width': '22em',
                        'margin': 'auto',
                        'padding': '10px',
                        }
                    ),

                    # radio
                    html.Div(style={
                        'margin': 'auto',
                        'padding': '10px',
                        'color': colors['text'],
                        'fontSize': '15px',
                        'padding-left': '7%'
                    },
                        children=[
                                dcc.RadioItems(
                                id='vis_input',
                                options=[
                                    {'label': 'Positive tweets', 'value': 'Positive'},
                                    {'label': 'Negative tweets', 'value': 'Negative'},
                                ],
                                value='Positive',
                                labelStyle={
                                    'display': 'inline-block',
                                    'width': '10.5em',
                                    'line-height': '0.5em'
                                    }
                                )
                            ]
                        )
                    ]
                )
            ]    
        ),
        
    html.Div(style={
        'margin': '10px',
        'width': '67%',
        'top': '-5px',
        'padding': '10px',
        'float': 'left',
        'clear': 'left'
    },
        children=[
            html.P(id="info-text",
                   style={
                       'padding-top': '1.5%',
                       'padding-left': '5%',
                       'color': colors['text'],
                       }
                    )
                ]
            ),
        ]
    )


@app.callback([Output("visualization", "children"),
               Output("info-text", "children")],
                Input("vis_input", "value"),
                Input("my-slider", "value"))
def update_wordcloud(vis_input, slider_input):
    info_text = ""
    if slider_input == 1:
        df = pd.read_csv('C:/Users/Rnowa/Documents/interactive-data-visualization-webapp/data/LUNA1-USD.csv')

        fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
        fig.update_layout(title_text='Price of the LUNA cryptocurrency', title_x=0.5)
        info_text = text1
        
    elif slider_input == 2 and 'Positive' in vis_input:
        fig = px.bar(pd_pos, x='keywords', y='freq', title="Keywords from positive tweets")
        fig.update_layout(xaxis_tickangle=-45, title_x=0.5)
        info_text = text2
    
    elif slider_input == 2 and 'Negative' in vis_input:
        fig = px.bar(pd_neg, x='keywords', y='freq', title="Keywords from negative tweets")
        fig.update_layout(xaxis_tickangle=-45, title_x=0.5)
        info_text = text2
        
    elif slider_input == 3 and 'Positive' in vis_input:
        img = np.array(Image.open("C:/Users/Rnowa/Documents/interactive-data-visualization-webapp/plots/pos.png"))
        fig = px.imshow(img)
        fig.update_layout(coloraxis_showscale=False)
        fig.update_xaxes(showticklabels=False)
        fig.update_yaxes(showticklabels=False)
        fig.update_layout(title_text='Keywords from positive tweets', title_x=0.5)
        info_text = text3
        
    elif slider_input == 3 and 'Negative' in vis_input:
        img = np.array(Image.open("C:/Users/Rnowa/Documents/interactive-data-visualization-webapp/plots/neg.png"))
        fig = px.imshow(img)
        fig.update_layout(coloraxis_showscale=False)
        fig.update_xaxes(showticklabels=False)
        fig.update_yaxes(showticklabels=False)
        fig.update_layout(title_text='Keywords from negative tweets', title_x=0.5)
        info_text = text3
    
    return dcc.Graph(figure=fig), info_text
        

In [ ]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [14/Jun/2022 18:33:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2022 18:33:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2022 18:33:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2022 18:33:44] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2022 18:33:44] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Jun/2022 18:33:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2022 18:33:44] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Jun/2022 18:33:44] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Jun/2022 18:33:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2022 18:33:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2022 18:34:06] "POST /_dash-update-component HTTP/1.1" 200